In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
class DataGeneration:
    
    # target_position = 0 (첫번째열이 정답데이터), target_position=-1 (마지막열이 정답데이터)
    def __init__(self, name, file_path, seperation_rate, target_position=-1):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        if (target_position == -1  or  target_position == 0):      
            self.target_position = target_position
        
        else:
            err_str = 'target_position must be -1 or 0'            
            raise Exception(err_str)    
            
    
    # print data target distribution 
    # str_of_kind : 'original data' or  'training data'  or  'test data'
    def print_target_distribution(self, data, str_of_kind='original data'):
        
        print('=======================================================================================================')
        
        target_data = data[ :, self.target_position ]
        
        # numpy.unique() 사용하여 loaded data target 분포 확인
        unique, counts = np.unique(target_data, return_counts=True)

        print('[DataGeneration]  ', str_of_kind, ' target value = ', dict(zip(unique, counts)).items())

        num_zeros = dict(zip(unique, counts))[0.0]  # key 0.0 에 대한 value 값 count 리턴
        num_ones = dict(zip(unique, counts))[1.0]  # key 1.0 에 대한 value 값 count 리턴

        print('[DataGeneration]  ', str_of_kind, ' zeros numbers = ', num_zeros, ', ratio = ', 100 * num_zeros / (data.shape[0]), ' %')
        print('[DataGeneration]  ', str_of_kind, ' ones numbers = ', num_ones, ', ratio = ', 100 * num_ones / (data.shape[0]), '%') 
    
        print('=======================================================================================================')
        
        
    # shuffle 기능을 이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기, 파일이 없는 경우 exception 발생

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))

        print("[DataGeneration]  loaded_data.shape = ", loaded_data.shape)
            
        # print the target distribution of original data 
        
        self.print_target_distribution(loaded_data, 'original data')
        
        
        # random.shuffle() 이용한 데이터 인덱스 분리 및 트레이닝/테스트 데이터 생성
        
        # 임시 저장 리스트
        training_data_list = []
        test_data_list = []

        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        #print("[DataGeneration]  total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

        # 전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num) ]

        # random.shuffle 을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list)  # 전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다

        # test data 를 위한 인덱스는 total_data_index_list 로뷰터 앞에서 분리비율(seperation_rate)의 데이터 인덱스
        test_data_index_list = total_data_index_list[ 0:test_data_num ]

        #print("[DataGeneration]  length of test_data_index_list = ", len(test_data_index_list))

        # training data 를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num: ]

        #print("[DataGeneration]  length of training_data_index_list = ", len(training_data_index_list))

        # training data 구성
        for training_data_index in training_data_index_list:
    
            training_data_list.append(loaded_data[training_data_index])

        # test data 구성
        for test_data_index in test_data_index_list:
    
            test_data_list.append(loaded_data[test_data_index])

        # generate training data from training_data_list using np.arrya(...)
        training_data = np.array(training_data_list)

        # generate test data from test_data_list using np.arrya(...)
        test_data = np.array(test_data_list)

        # verification shape
        #print("[DataGeneration]  training_data.shape = ", training_data.shape)
        #print("[DataGeneration]  test_data.shape = ", test_data.shape)

        # print target distribution of generated data 
        
        self.print_target_distribution(training_data, 'training data')
        
        self.print_target_distribution(test_data, 'test data')
        
        
        # save training & test data (.csv)
        training_data_save_path = './' + self.name + '_training_data.csv'
        test_data_save_path = './' + self.name + '_test_data.csv'
        
        # 저장공간이 없거나 파일 write 실패시 exception 발생
        try:
            np.savetxt(training_data_save_path, training_data, delimiter=',')
            np.savetxt(test_data_save_path, test_data, delimiter=',')
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))
        
        return training_data, test_data

In [3]:
# DataGeneration 객체 생성
seperation_rate = 0.4
data_obj = DataGeneration('Diabetes', './(191117)diabetes.csv', seperation_rate)  

# training_data, test_data 생성
(training_data, test_data) = data_obj.generate()

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

DataGeneration object is created !!
[debug. before data normalization] loaded_data[0] =  [-0.294118    0.487437    0.180328   -0.292929    0.          0.00149028
 -0.53117    -0.0333333   0.        ]
loaded_data.shape =  (759, 9)
total_data_num =  759 , test_data_num =  227
length of test_data_index_list =  227
length of training_data_index_list =  532
training_data.shape =  (532, 9)
test_data.shape =  (227, 9)
training_data.shape =  (532, 9)
test_data.shape =  (227, 9)


In [4]:
training_x_data = training_data[ :, 0:-1]
training_t_data = training_data[ :, [-1]]

print("training_x_data.shape = ", training_x_data.shape)
print("training_t_data.shape = ", training_t_data.shape)

test_x_data = test_data[ :, 0:-1]
test_t_data = test_data[ :, [-1]]

print("test_x_data.shape = ", test_x_data.shape)
print("test_x_data.shape = ", test_x_data.shape)

training_x_data.shape =  (532, 8)
training_t_data.shape =  (532, 1)
test_x_data.shape =  (227, 8)
test_x_data.shape =  (227, 8)


In [5]:
X = tf.placeholder(tf.float32, [None, 8])  # 8개 입력노드
T = tf.placeholder(tf.float32, [None, 1])  # 1개 정답노드

W = tf.Variable(tf.random_normal([8, 1]))  # 8X1 가중치노드
b = tf.Variable(tf.random_normal([1]))     # 1개 바이어스 노드

In [6]:
z = tf.matmul(X, W) + b  # 선형회귀 값 z

y = tf.sigmoid(z)    # 시그모이드로 계산 값

# 손실함수는 Cross-Entropy 
loss = -tf.reduce_mean( T*tf.log(y) + (1-T)*tf.log(1-y) ) 

In [7]:
learning_rate = 0.01    # 학습율

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [8]:
# 정확성 검사, True if y > 0.5 else False

predicted = tf.cast(y > 0.5, dtype=tf.float32)  


# predicted 와 T 같으면 True 를 리턴하므로 cast 에 의해서 1로 강제 변환, 
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype=tf.float32))

In [10]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(20001):
      
        loss_val, _ = sess.run([loss, train], feed_dict={X: training_x_data, T: training_t_data})    
        
        
        if step % 500 == 0:
            print("step = ", step, ", loss_val = ", loss_val)             
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X: test_x_data, T: test_t_data})
    
    
    print("\ny_val.shape = ", y_val.shape, ", predicted_val = ", predicted_val.shape)    
    print("\nAccuracy = ", accuracy_val)

step =  0 , loss_val =  0.6965248
step =  500 , loss_val =  0.6352679
step =  1000 , loss_val =  0.6014237
step =  1500 , loss_val =  0.57615703
step =  2000 , loss_val =  0.55715686
step =  2500 , loss_val =  0.5427557
step =  3000 , loss_val =  0.5317255
step =  3500 , loss_val =  0.52317756
step =  4000 , loss_val =  0.5164724
step =  4500 , loss_val =  0.5111499
step =  5000 , loss_val =  0.5068766
step =  5500 , loss_val =  0.5034091
step =  6000 , loss_val =  0.50056773
step =  6500 , loss_val =  0.49821812
step =  7000 , loss_val =  0.49625918
step =  7500 , loss_val =  0.4946134
step =  8000 , loss_val =  0.4932212
step =  8500 , loss_val =  0.492036
step =  9000 , loss_val =  0.4910213
step =  9500 , loss_val =  0.49014753
step =  10000 , loss_val =  0.48939174
step =  10500 , loss_val =  0.4887348
step =  11000 , loss_val =  0.48816136
step =  11500 , loss_val =  0.48765886
step =  12000 , loss_val =  0.48721674
step =  12500 , loss_val =  0.4868264
step =  13000 , loss_val =